In [13]:
# from flask import Flask,g

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv

from module.db import Database
from module.rag import RAGChatbot

import os
load_dotenv()

True

In [5]:
DATABASE_FILE = 'database.db'
EMBEDDING_NAME = "LazarusNLP/all-indobert-base-v2"
LLM_NAME = "gemini-2.5-flash"
VECTOR_STORE_DIR = "vectorstore_chroma_db1"
COLLECTION_NAME = "grape_vector_store"

chatbot_model = ChatGoogleGenerativeAI(
    model=LLM_NAME,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

if os.path.exists(VECTOR_STORE_DIR):
    print(f"Vector store directory '{VECTOR_STORE_DIR}' found.")
    try:
        embedding_model = HuggingFaceEmbeddings(
            model_name=EMBEDDING_NAME,
            model_kwargs={"device": "cuda"},   # ubah ke "cuda" jika pakai GPU
            encode_kwargs={'normalize_embeddings': True}
        )

        vector_store = Chroma(
            persist_directory=VECTOR_STORE_DIR,
            embedding_function=embedding_model,
            collection_name=COLLECTION_NAME
        )

        rag_chatbot = RAGChatbot(chatbot_model, vector_store)
        print("RAG chatbot initialized successfully.")
        print(f"Loaded existing vectorstore with {vector_store._collection.count()} documents")
    except Exception as e:
        print(f"Error initializing RAG chatbot: {e}")
        rag_chatbot = None
else:
    print(f"Vector store directory '{VECTOR_STORE_DIR}' not found. RAG chatbot will not be available.")

Vector store directory 'vectorstore_chroma_db1' found.
RAG chatbot initialized successfully.
Loaded existing vectorstore with 283 documents


In [ ]:
# app = Flask(__name__)

# def get_db():
#     """
#     Membuka koneksi database baru jika belum ada untuk request saat ini.
#     """
#     if 'db' not in g:
#         g.db = Database(DATABASE_FILE)
#         g.db.init_db() # Pastikan tabel ada
#     return g.db

# @app.teardown_appcontext
# def close_db(e=None):
#     """
#     Menutup koneksi database secara otomatis setelah request selesai.
#     """
#     db = g.pop('db', None)
#     if db is not None:
#         db.close()

In [14]:
user_question = "Apa penyakit yang disebabkan oleh jamur Cercospora spp. pada tanaman anggur?"
jawab = rag_chatbot.hybrid_search(user_question)

TypeError: RAGChatbot.hybrid_search() missing 1 required positional argument: 'chat_history'

In [16]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.8.0+cu128
CUDA available: True
CUDA version: 12.8
